# Workshop: Multi-Agent система поиска лучшего курса валют

## Добро пожаловать на 30-минутный workshop по созданию мульти-агентных систем!

### Что мы будем изучать:
- Фреймворк **smolagents** от Hugging Face
- Создание кастомных инструментов для агентов
- Интеграцию с внешними API (Serper.dev, ЦБ РФ)
- Координацию работы между несколькими специализированными агентами

### Сценарий:
Создадим систему из 4 агентов, которые помогут найти лучший курс обмена USD на рубли:
1. **CurrencyAgent** - получает официальный курс ЦБ РФ
2. **LocationAgent** - находит банки рядом с пользователем
3. **BankRatesAgent** - собирает курсы обмена в банках
4. **DecisionAgent** - анализирует и рекомендует лучший вариант

## Шаг 1: Установка и импорты (5 минут)

Убедитесь, что у вас установлены все необходимые зависимости:
```bash
pip install -r requirements.txt
```

In [1]:
# Импорты
import os
from dotenv import load_dotenv
from smolagents import CodeAgent
from smolagents.models import OpenAIModel
from cloud_ru_model import CloudRuModel, CloudRuModelWithFallback, test_cloud_ru_connection
import warnings
warnings.filterwarnings('ignore')

# Загружаем переменные окружения
load_dotenv()

# Импортируем наши кастомные инструменты
from custom_tools import (
    SerperSearchTool, 
    CBRCurrencyTool, 
    UserInputTool, 
    AddressToCoordsTool,
)

print("✅ Все модули импортированы успешно!")

✅ Все модули импортированы успешно!


In [ ]:
amount = float(input("Введите сумму для покупки валюты: ").replace(" ", "").replace(",", "."))


## Шаг 2: Настройка API ключей

**Важно:** Создайте файл `.env` на основе `.env.example` и заполните ваши API ключи:
- **Cloud.ru Foundation Models API** - основной провайдер LLM (российский)
- Serper.dev API ключ для поиска в Google
- OpenAI API ключ (опционально, как fallback)

In [2]:
# Проверяем наличие API ключей
def check_api_keys():
    # Проверяем Cloud.ru API (основной)
    cloud_ru_api = os.getenv("CLOUD_RU_API_KEY") or os.getenv("API_KEY")
    
    # Проверяем дополнительные API
    other_keys = {
        "SERPER_API_KEY": os.getenv("SERPER_API_KEY"),
        "OPENAI_API_KEY": os.getenv("OPENAI_API_KEY")  # Опционально
    }
    
    print("🏢 Cloud.ru Foundation Models API (основной провайдер):")
    if cloud_ru_api:
        print(f"  ✅ API ключ настроен (начинается с {cloud_ru_api[:10]}...)")
        cloud_ru_ok = True
    else:
        print(f"  ❌ API ключ НЕ НАСТРОЕН! Установите CLOUD_RU_API_KEY или API_KEY")
        cloud_ru_ok = False
    
    print("\n🔧 Дополнительные API:")
    for key_name, key_value in other_keys.items():
        if key_value:
            print(f"  ✅ {key_name}: настроен")
        else:
            status = "НЕ НАСТРОЕН" if key_name == "SERPER_API_KEY" else "НЕ НАСТРОЕН (опционально)"
            print(f"  ❌ {key_name}: {status}")
    
    serper_ok = bool(other_keys["SERPER_API_KEY"])
    
    return cloud_ru_ok and serper_ok, cloud_ru_ok, serper_ok

all_keys_ok, cloud_ru_ok, serper_ok = check_api_keys()
print(f"\nГотовность к работе: {'✅' if all_keys_ok else '⚠️ частично' if cloud_ru_ok else '❌'}")

# Тестируем подключение к Cloud.ru
if cloud_ru_ok:
    print("\n🧪 Тестирование подключения к Cloud.ru API...")
    cloud_connection_ok = test_cloud_ru_connection()
else:
    cloud_connection_ok = False
    print("\n❌ Невозможно протестировать Cloud.ru API - отсутствует ключ")

🏢 Cloud.ru Foundation Models API (основной провайдер):
  ✅ API ключ настроен (начинается с YWYyNGYzZD...)

🔧 Дополнительные API:
  ✅ SERPER_API_KEY: настроен
  ✅ OPENAI_API_KEY: настроен

Готовность к работе: ✅

🧪 Тестирование подключения к Cloud.ru API...
🧪 Тестируем подключение к Cloud.ru API...
✅ Подключение успешно! Ответ: да
📊 Модель: zai-org/GLM-4.5
🔗 URL: https://foundation-models.api.cloud.ru/v1


## Шаг 3: Создание специализированных агентов (15 минут)

### 3.1 CurrencyAgent - Агент для получения официального курса валют

In [3]:
# Создаем модель для наших агентов
# Используем Cloud.ru с моделью GLM-4.5 (zai-org)

if cloud_ru_ok:
    print("🏢 Используем Cloud.ru Foundation Models API")
    print("🤖 Модель: zai-org/GLM-4.5 (355B параметров, оптимизирована для агентов)")
    try:
        model = CloudRuModel(
            model_name="zai-org/GLM-4.5",  # Используем GLM-4.5 вместо GPT
            temperature=0.5,
            max_tokens=5000,
            top_p=0.95,
            presence_penalty=0
        )
        print("✅ Модель GLM-4.5 инициализирована (без fallback)")
    except Exception as e:
        print(f"❌ Ошибка инициализации GLM-4.5: {e}")
        raise
else:
    print("❌ Cloud.ru API недоступен")
    raise ValueError("Требуется настройка Cloud.ru API в файле .env")

print(f"📊 Информация о модели: {getattr(model, 'get_model_info', lambda: {'provider': 'Unknown'})()}")

# Агент для получения официального курса валют
currency_agent = CodeAgent(
    tools=[CBRCurrencyTool()],
    model=model
)

print("✅ CurrencyAgent создан с GLM-4.5!")

# Тестируем агента
print("\n🧪 Тестирование CurrencyAgent с GLM-4.5:")
result = currency_agent.run("Ты - CurrencyAgent, специалист по валютным курсам. Получи актуальный курс доллара США к рублю от ЦБ РФ и предоставь точную информацию о курсе и дате его установления. Отвечай на русском языке.")
print(result)

🏢 Используем Cloud.ru Foundation Models API
🤖 Модель: zai-org/GLM-4.5 (355B параметров, оптимизирована для агентов)
✅ Модель GLM-4.5 инициализирована (без fallback)
📊 Информация о модели: {'provider': 'Cloud.ru', 'model_name': 'zai-org/GLM-4.5', 'base_url': 'https://foundation-models.api.cloud.ru/v1', 'temperature': 0.5, 'max_tokens': 5000, 'top_p': 0.95}
✅ CurrencyAgent создан с GLM-4.5!

🧪 Тестирование CurrencyAgent с GLM-4.5:


╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Ты - CurrencyAgent, специалист по валютным курсам. Получи актуальный курс доллара США к рублю от ЦБ РФ и        │
│ предоставь точную информацию о курсе и дате его установления. Отвечай на русском языке.                         │
│                                                                                                                 │
╰─ CloudRuModel - None ───────────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  currency_info = cbr_currency()                                                                                   
  print(currency_info)                                                                                             
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Официальный курс ЦБ РФ на 2025-08-23:
USD/RUB: 80.7498 руб.

Out: None

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("Официальный курс доллара США к рублю по данным Центрального Банка РФ на 23 августа 2025 года       
  составляет 80.7498 рубля за 1 доллар США.")                                                                      
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: Официальный курс доллара США к рублю по данным Центрального Банка РФ на 23 августа 2025 года 
составляет 80.7498 рубля за 1 доллар США.

[Step 2: Duration 1.25 seconds]

Официальный курс доллара США к рублю по данным Центрального Банка РФ на 23 августа 2025 года составляет 80.7498 рубля за 1 доллар США.


### 3.2 LocationAgent - Агент для поиска банков по местоположению

In [11]:
# Агент для работы с местоположением и поиском банков
location_agent = CodeAgent(
    tools=[SerperSearchTool(), AddressToCoordsTool()],
    model=model,
    name="location_agent",
    description="Отвечает за поиск банков рядом с адресом пользователя и получения их географических координат",
)

print("✅ LocationAgent создан!")


✅ LocationAgent создан!


### 3.3 BankRatesAgent - Агент для получения курсов в банках

In [12]:
# Агент для получения курсов валют в банках
from smolagents import VisitWebpageTool
visit_webpage_tool = VisitWebpageTool()
bank_rates_agent = CodeAgent(
    tools=[visit_webpage_tool], 
    model=model,
    name="bank_rates_agent",
    description="Отвечает за поиск курса валют в банках",
)

banki_ros_url = 'https://bankiros.ru/currency/usd?page='
banki_ru_url = 'https://www.banki.ru/products/currency/cash/usd/moskva/'

print("✅ BankRatesAgent создан!")

✅ BankRatesAgent создан!


In [ ]:
# Тестируем агента
print("\n🧪 Тестирование BankRatesAgent:")
result = bank_rates_agent.run(f"""Ты - BankRatesAgent, специалист по банковским курсам валют. 
Получи курсы покупки и продажи USD в различных банках, используй {banki_ros_url}. Отвечай на русском языке.""")
print(result)

### 3.4 DecisionAgent - Агент для анализа и принятия решений

In [ ]:
# Агент для анализа и принятия решений


manager_agent = CodeAgent(
    tools=[UserInputTool()],  # Этот агент работает только с входом пользователя
    model=model,
    managed_agents=[location_agent, bank_rates_agent],
)

result = manager_agent.run(f"""Ты - ManagerAgent. Помощник выбора банка рядом с пользователем для обмена курса валют.
Предоставляй четкие рекомендации: лучший банк для покупки валюты (наименьший курс продажи), 
лучший банк для продажи валюты (наибольший курс покупки), общие выводы и советы. 
Отвечай на русском языке. Руководствуйся следующей логикой что если купить 100$ то лучше сделать это поближе к дому, 
а если 5000$ нужно купить то нужен выгодный курс и можно подальше от дома до 10 километров расстояния.
Используй {banki_ros_url}, {banki_ru_url} для поиска курса валют""")

print("✅ DecisionAgent создан!")
print("\n🎉 Все агенты готовы к работе!")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Ты - ManagerAgent. Помощник выбора банка рядом с пользователем для обмена курса валют.                          │
│ Предоставляй четкие рекомендации: лучший банк для покупки валюты (наименьший курс продажи),                     │
│ лучший банк для продажи валюты (наибольший курс покупки), общие выводы и советы.                                │
│ Отвечай на русском языке. Руководствуйся следующей логикой что если купить 100$ то лучше сделать это поближе к  │
│ дому,                                                                                                           │
│ а если 5000$ нужно купить то нужен выгодный курс и можно подальше от дома до 10 километров расстояния.          │
│ Используй https://bankiros.ru/currency/usd?page=, https://www.banki.ru/products/currency/cash/usd/moskva/ для   │
│ поиска курса валют                                                                                              │
│                                                                                                                 │
╰─ CloudRuModel - None ───────────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  user_address = user_input("Пожалуйста, введите ваш адрес для поиска ближайших банков: ")                         
  print(f"Адрес пользователя: {user_address}")                                                                     
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Адрес пользователя: москва, площадь гагарина

Out: None

[Step 1: Duration 17.82 seconds]

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 5 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("""                                                                                                 
  # Рекомендации по выбору банка для обмена валюты USD                                                             
                                                                                                                   
  ## Лучшие банки для обмена валюты                                                                                
                                                                                                                   
  ### Для покупки USD (наиболее выгодный курс продажи):                                                            
  **Камкомбанк** - курс продажи: 80.19 руб. за 1 USD                                                               
  - Адрес: Москва, площадь Победы, 2, корп.                                                                        
  - Дополнительная информация: Бесплатный резерв суммы до конца дня. Комиссия за обмен валюты 200Р.                
                                                                                                                   
  ### Для продажи USD (наиболее выгодный курс покупки):                                                            
  **Камкомбанк** - курс покупки: 80.11 руб. за 1 USD                                                               
  - Адрес: Москва, площадь Победы, 2, корп.                                                                        
  - Дополнительная информация: Бесплатный резерв суммы до конца дня. Комиссия за обмен валюты 200Р.                
                                                                                                                   
  ## Общие выводы                                                                                                  
                                                                                                                   
  Камкомбанк предлагает самые выгодные условия как для покупки, так и для продажи USD. У этого банка наименьший    
  спред (разница между курсами покупки и продажи) - всего 0.08 руб., что делает его наиболее привлекательным для   
  обмена валюты.                                                                                                   
                                                                                                                   
  ## Рекомендации в зависимости от суммы обмена                                                                    
                                                                                                                   
  ### Для покупки 100$:                                                                                            
  Рекомендую обратиться в **СберБанк** (отделение по адресу: Москва, ул. Вавилова, 3, этаж 2 или ул. Вавилова,     
  19), который находится ближе всего к площади Гагарина. Хотя курс продажи у СберБанка (82 руб.) выше, чем у       
  Камкомбанка, для небольшой суммы разница составит всего около 180 руб., что компенсируется удобством             
  расположения.                                                                                                    
                                                                                                                   
  ### Для покупки 5000$:                                                                                           
  Рекомендую обратиться в **Камкомбанк** (адрес: Москва, площадь Победы, 2, корп.), несмотря на то, что он         
  находится дальше от площади Гагарина. При такой сумме разница в курсе позволит сэкономить около 9000 руб. по     
  сравнению с СберБанком, что оправдывает поездку на большее расстояние.                                           
                                                         

Out - Final answer: 
# Рекомендации по выбору банка для обмена валюты USD

## Лучшие банки для обмена валюты

### Для покупки USD (наиболее выгодный курс продажи):
**Камкомбанк** - курс продажи: 80.19 руб. за 1 USD
- Адрес: Москва, площадь Победы, 2, корп.
- Дополнительная информация: Бесплатный резерв суммы до конца дня. Комиссия за обмен валюты 200Р.

### Для продажи USD (наиболее выгодный курс покупки):
**Камкомбанк** - курс покупки: 80.11 руб. за 1 USD
- Адрес: Москва, площадь Победы, 2, корп.
- Дополнительная информация: Бесплатный резерв суммы до конца дня. Комиссия за обмен валюты 200Р.

## Общие выводы

Камкомбанк предлагает самые выгодные условия как для покупки, так и для продажи USD. У этого банка наименьший спред
(разница между курсами покупки и продажи) - всего 0.08 руб., что делает его наиболее привлекательным для обмена 
валюты.

## Рекомендации в зависимости от суммы обмена

### Для покупки 100$:
Рекомендую обратиться в **СберБанк** (отделение по адресу: Москва, ул. Вавилова, 3, этаж 2 или ул. Вавилова, 19), 
который находится ближе всего к площади Гагарина. Хотя курс продажи у СберБанка (82 руб.) выше, чем у Камкомбанка, 
для небольшой суммы разница составит всего около 180 руб., что компенсируется удобством расположения.

### Для покупки 5000$:
Рекомендую обратиться в **Камкомбанк** (адрес: Москва, площадь Победы, 2, корп.), несмотря на то, что он находится 
дальше от площади Гагарина. При такой сумме разница в курсе позволит сэкономить около 9000 руб. по сравнению с 
СберБанком, что оправдывает поездку на большее расстояние.

## Дополнительные советы

1. Перед посещением банка рекомендуется уточнять актуальность курсов по телефону.
2. Для обмена валюты необходимо иметь при себе паспорт.
3. Камкомбанк предлагает бесплатный резерв суммы до конца дня, что особенно важно при обмене крупных сумм.
4. Учитывайте возможные комиссии за обменные операции.
5. Курсы валют могут меняться в течение дня, поэтому при планировании крупной операции стоит следить за динамикой.

Официальный курс Центрального Банка РФ на 22.08.2025 составляет 80.2548 рубля за 1 USD.

[Step 5: Duration 6.80 seconds]

✅ DecisionAgent создан!

🎉 Все агенты готовы к работе!


## Заключение и дальнейшее развитие

### Что мы изучили:
- ✅ Фреймворк **smolagents** от Hugging Face
- ✅ Создание кастомных инструментов (Tools) для агентов
- ✅ Интеграцию с внешними API (Serper.dev, ЦБ РФ)
- ✅ Координацию работы между специализированными агентами
- ✅ Построение end-to-end решения с мульти-агентной архитектурой

### Возможные улучшения:
1. **Добавить больше источников данных** - другие поисковые API, банковские API
2. **Улучшить парсинг результатов** - более точное извлечение информации о банках
3. **Добавить геолокацию** - автоматическое определение местоположения пользователя
4. **Кэширование данных** - сохранение результатов для повторного использования
5. **Визуализация** - графики курсов, карты с банками
6. **Мониторинг курсов** - отслеживание изменений в реальном времени

### Дополнительные ресурсы:
- [Документация smolagents](https://huggingface.co/docs/smolagents/index)
- [Serper.dev API Documentation](https://serper.dev/)
- [ЦБ РФ API](https://www.cbr.ru/development/dws/)

---

**Спасибо за участие в workshop! 🎉**

Теперь вы знаете основы создания мульти-агентных систем с smolagents!

## Бонус: Экспериментируйте дальше!

Попробуйте модифицировать агентов или добавить новые возможности:

In [ ]:
# Пример: создайте своего собственного агента
# experimental_agent = CodeAgent(
#     tools=[...],
#     model=model,
#     system_prompt="Ваш промпт..."
# )

print("💡 Место для ваших экспериментов!")
print("Попробуйте:")
print("- Изменить промпты агентов")
print("- Добавить новые инструменты")
print("- Создать агента для другой валюты")
print("- Добавить сравнение с курсами онлайн-обменников")